# Exploring Physical Properties of an Electrolye

This notebook is intended to serve as a tutorial of how to setup and execute an end-to-end a bespoke HTMD workflow using atomate2-openmm. The end-to-end example will be in terms of exploring the physical properties of a constrained randok sample of electrolyte formulations as part of a developing lithium ion batteries.

Let's first frame the problem. Suppose we are looking to develop an electrolyte formulaton for a lithium ion battery. Basic domain knowledge on electrolyte formulating suggest that a battery electrolyte can typically consist of components tabulated below with concentrations given in moles.

| Component                            | Concentration (moles)     | Concentration (number of molecules) | SMILES String          |
|--------------------------------------|----------------------------|-------------------------------------|------------------------|
| Lithium Hexafluorophosphate (LiPF6)  | 0.0010 - 0.0025           | 50 - 100                            | [Li+].F[P-](F)(F)(F)(F)F |
| Ethylene Carbonate (EC)              | 0.0100 - 0.0250           | 500 - 1000                          | CC(=O)OCC               |
| Dimethyl Carbonate (DMC)             | 0.0100 - 0.0250           | 500 - 1000                          | CC(=O)OC(=O)C            |
| Diethyl Carbonate (DEC)              | 0.0040 - 0.0100           | 200 - 500                           | CCOC(=O)CC               |
| Vinylene Carbonate (VC)              | 0.0002 - 0.0010           | 10 - 100                            | C1C(=O)OC=C1             |
| Fluoroethylene Carbonate (FEC)       | 0.0002 - 0.0010           | 10 - 100                            | C1C(=O)OCCF1             |


What we would like to do is a constrained uniform random sample of 10 random formulations electrolyte recipes and run them through an HTMD workflow using atomate2-openmm and analyze the results (for property X). To begin this process we first need to convert the collection of simulation we would like to run into the XML files that are used as simulation input. Creating XML files is cumbersome, so we can utilize pymatgen-io-openmm to generate these XML files from basic information like chemical structure and the number of molcules to include.

- 10 non-random formulations in a specific order
- Load in data from completed simulation. Meaningful simulations need to run

For convenience, let's define a helper function that will randomly sample an electroltye formulation from the 6 dimensional chemical space captured by the table above.

In [56]:
from pymatgen.io.openmm.generators import OpenMMSolutionGen
import random

openmm_sol_gen = OpenMMSolutionGen(default_charge_method="mmff94")

def generate_random_openmm_set():
    openmm_set = openmm_sol_gen.get_input_set(
        input_mol_dicts=[
    #         {"name": "LiPF6", "smile": "[Li+].F[P-](F)(F)(F)(F)F", "count": random.randint(50, 100)},
            {"name": "EC", "smile": "CC(=O)OCC", "count": random.randint(500, 1000)},
            {"name": "DMC", "smile": "CC(=O)OC(=O)C", "count": random.randint(500, 1000)},
            {"name": "DEC", "smile": "CCOC(=O)CC", "count": random.randint(200, 500)},
            {"name": "VC", "smile": "C1C(=O)OC=C1", "count": random.randint(10, 100)},
    #         {"name": "FEC", "smile": "C1C(=O)OCCF1", "count": random.randint(10, 100)},
        ],
        density=1.0,
    )
    return openmm_set

Now let's generate the atomate2-openmm objects to as part of the HTMD workflow.

In [59]:
from atomate2_openmm.flows.production_maker import ProductionMaker
from atomate2_openmm.flows.anneal_maker import AnnealMaker
from atomate2_openmm.jobs.energy_minimization_maker import EnergyMinimizationMaker
from atomate2_openmm.jobs.nvt_maker import NVTMaker
from atomate2_openmm.jobs.npt_maker import NPTMaker
from atomate2_openmm.jobs.temp_change_maker import TempChangeMaker

In [61]:
energy_minimization_maker = EnergyMinimizationMaker()

In [62]:
npt_maker = NPTMaker(
    steps=100,
    state_reporter_interval=10,
    dcd_reporter_interval=10,
)

In [63]:
anneal_maker = AnnealMaker(
    raise_temp_maker=TempChangeMaker(
        steps=1000,
        temp_steps=10,
        final_temp=700,
        state_reporter_interval=0,
        dcd_reporter_interval=0,
    )
)

In [60]:
from maggma.stores import MongoURIStore
from maggma.stores.aws import S3Store
from maggma.stores import MemoryStore

In [58]:
from jobflow import run_locally
from jobflow import JobStore

# Scratch Work

In [42]:
from openff.toolkit.utils.toolkits import (
    OpenEyeToolkitWrapper, 
    RDKitToolkitWrapper, 
    AmberToolsToolkitWrapper, 
    ToolkitRegistry,
)
from openff.toolkit import Molecule

In [43]:
molecule = Molecule.from_smiles(smiles="[Li+].F[P-](F)(F)(F)(F)F")

In [45]:
molecule.assign_partial_charges(partial_charge_method="mmff94", toolkit_registry=RDKitToolkitWrapper())

AttributeError: 'NoneType' object has no attribute 'GetMMFFPartialCharge'

In [50]:
molecule.generate_conformers(n_conformers=1, toolkit_registry=RDKitToolkitWrapper())

[10:11:06] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[10:11:06] UFFTYPER: Unrecognized charge state for atom: 1
[10:11:06] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[10:11:06] UFFTYPER: Unrecognized charge state for atom: 1


ConformerGenerationError: RDKit conformer generation failed.

In [ ]:
Molecule.from_smiles(smiles="[Li+].F[P-](F)(F)(F)(F)F")

In [ ]:
Molecule.from_smiles(smiles="CC(=O)OCC")

In [ ]:
Molecule.from_smiles(smiles="CC(=O)OC(=O)C")

In [ ]:
Molecule.from_smiles(smiles="CCOC(=O)CC")

In [ ]:
Molecule.from_smiles(smiles="CC(=O)OC(=O)C")

In [ ]:
Molecule.from_smiles(smiles="C1C(=O)OC=C1")

In [ ]:
Molecule.from_smiles(smiles="C1C(OC(=O)O1)F")